# Early Church Martyrdoms (1st-5th Centuries)

This notebook analyzes recorded martyrdoms in the early Christian church across the first five centuries with interactive visualizations and geographic analysis.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

## Data Collection

Create a comprehensive dataset of recorded martyrdoms from the first 5 centuries of the church with geographic coordinates.

In [ ]:
# Create the martyrdoms dataset with geographic coordinates
martyrdoms_data = {
    'Year': [34, 43, 54, 60, 64, 68, 72, 81, 95, 107, 155, 165, 177, 203, 250, 258, 303, 313, 335, 451],
    'Deaths': [1, 1, 1, 1, 100, 1, 3, 100, 1, 1, 1, 7, 48, 5, 100, 3, 1000, 0, 100, 100],
    'Emperor': ['Tiberius', 'Claudius', 'Nero', 'Nero', 'Nero', 'Nero', 'Vespasian', 'Domitian', 'Domitian', 'Trajan', 
                'Antoninus Pius', 'Marcus Aurelius', 'Marcus Aurelius', 'Septimius Severus', 'Decius', 'Valerian', 
                'Diocletian', 'Constantine', 'Constantine II', 'Theodosius II'],
    'Event': ['Stephen', 'James the Greater', 'Philip the Apostle', 'Matthew the Apostle', 
              "Nero's Persecution", 'Mark the Apostle', 'Jude, Simon, Thomas (apostles)', 
              "Domitian's Persecution", 'John (exile)', 'Ignatius of Antioch', 'Polycarp', 
              'Justin Martyr and companions', 'Lyon Persecution', 'Perpetua & Felicity', 
              'Decius Persecution', 'Sixtus II & Lawrence', 'Diocletian Great Persecution', 
              'Edict of Milan (end of persecution)', "Shapur II's Persecution", 'Battle of Avarayr (Armenia)'],
    'Location': ['Jerusalem', 'Jerusalem', 'Hieropolis', 'Ethiopia', 'Rome', 'Alexandria', 
                 'Jerusalem/Edessa/India', 'Roman Empire', 'Patmos/Ephesus', 'Rome', 'Smyrna', 
                 'Rome', 'Lyon (Gaul)', 'Carthage', 'Roman Empire', 'Rome/Carthage', 
                 'Roman Empire', 'Roman Empire', 'Persian Empire', 'Armenia'],
    'Latitude': [31.778, 31.778, 37.745, 9.145, 41.902, 31.200, 31.778, 37.5, 37.5, 41.902, 38.640, 
                 41.902, 45.764, 36.753, 37.5, 41.902, 37.5, 37.5, 32.5, 40.0],
    'Longitude': [35.234, 35.234, 25.736, 40.500, 12.496, 29.900, 35.234, 15.0, 27.0, 12.496, 26.240, 
                  12.496, 4.835, 3.059, 15.0, 12.496, 15.0, 15.0, 53.7, 45.0]
}

df = pd.DataFrame(martyrdoms_data)
df['Period'] = pd.cut(df['Year'], bins=[0, 100, 200, 300, 400, 500], 
                       labels=['1st Century', '2nd Century', '3rd Century', '4th Century', '5th Century'])

print(df.to_string(index=False))
print(f"\nTotal estimated martyrs: {df['Deaths'].sum():,}")

 Year  Deaths           Emperor                               Event               Location  Latitude  Longitude      Period
   34       1          Tiberius                             Stephen              Jerusalem    31.778     35.234 1st Century
   43       1          Claudius                   James the Greater              Jerusalem    31.778     35.234 1st Century
   54       1              Nero                  Philip the Apostle             Hieropolis    37.745     25.736 1st Century
   60       1              Nero                 Matthew the Apostle               Ethiopia     9.145     40.500 1st Century
   64     100              Nero                  Nero's Persecution                   Rome    41.902     12.496 1st Century
   68       1              Nero                    Mark the Apostle             Alexandria    31.200     29.900 1st Century
   72       3         Vespasian      Jude, Simon, Thomas (apostles) Jerusalem/Edessa/India    31.778     35.234 1st Century
   81   

## Summary Statistics by Century

In [ ]:
# Summary statistics by century
century_summary = df.groupby('Period').agg({
    'Deaths': ['sum', 'mean', 'count']
}).round(2)

print("\nMartydoms by Century:")
print(century_summary)


Martydoms by Century:
            Deaths              
               sum    mean count
Period                          
1st Century    209   23.22     9
2nd Century     57   14.25     4
3rd Century    108   36.00     3
4th Century   1100  366.67     3
5th Century    100  100.00     1


/var/folders/nh/z1jf748x19108fww13nyfydw0000gp/T/ipykernel_16463/778530107.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  century_summary = df.groupby('Period').agg({


## Interactive Visualization: Martyrdoms Over Time

In [ ]:
# Create interactive Plotly bar chart
fig = go.Figure()

# Color code by century
colors = {
    '1st Century': '#0ea5e9',
    '2nd Century': '#06b6d4',
    '3rd Century': '#f97316',
    '4th Century': '#ef4444',
    '5th Century': '#8b5cf6'
}

for period in sorted(df['Period'].unique()):
    period_data = df[df['Period'] == period]
    fig.add_trace(go.Bar(
        x=period_data['Year'],
        y=period_data['Deaths'],
        name=str(period),
        marker_color=colors[str(period)],
        hovertemplate='<b>%{customdata[0]}</b><br>' +
                      'Year: %{x} AD<br>' +
                      'Deaths: %{y}<br>' +
                      'Emperor: %{customdata[1]}<br>' +
                      'Location: %{customdata[2]}<extra></extra>',
        customdata=period_data[['Event', 'Emperor', 'Location']].values
    ))

fig.update_layout(
    title='Early Church Martyrdoms (1st-5th Centuries)',
    xaxis_title='Year (AD)',
    yaxis_title='Estimated Number of Deaths',
    hovermode='x unified',
    template='plotly_white',
    height=600,
    barmode='group'
)

fig.show()
fig.write_html('01_martyrdoms_timeline.html')
print("Chart saved as '01_martyrdoms_timeline.html'")

Chart saved as '01_martyrdoms_timeline.html'


## Geographic Distribution: Interactive Map

In [ ]:
# Aggregate martyrdoms by location
location_summary = df.groupby(['Location', 'Latitude', 'Longitude']).agg({
    'Deaths': 'sum',
    'Event': lambda x: '<br>'.join(x)
}).reset_index().rename(columns={'Event': 'Events'})

print("Martyrdoms by Location:")
print(location_summary[['Location', 'Deaths']].sort_values('Deaths', ascending=False))

# Create scatter map with heatmap coloring
fig_map = go.Figure(data=go.Scattergeo(
    lon=location_summary['Longitude'],
    lat=location_summary['Latitude'],
    mode='markers',
    marker=dict(
        size=location_summary['Deaths'] / 10 + 5,
        color=location_summary['Deaths'],
        colorscale='Reds',
        showscale=True,
        colorbar=dict(title='Deaths'),
        line=dict(width=1, color='white'),
        opacity=0.8
    ),
    text=location_summary.apply(
        lambda x: f"<b>{x['Location']}</b><br>Deaths: {x['Deaths']}<br><br>" + 
                  x['Events'].replace('<br>', '<br>'),
        axis=1
    ),
    hovertemplate='%{text}<extra></extra>'
))

fig_map.update_layout(
    title='Geographic Distribution of Early Church Martyrdoms',
    geo=dict(
        projection_type='natural earth',
        showland=True,
        landcolor='rgb(243, 243, 243)',
        showcountries=True,
        countrycolor='rgb(204, 204, 204)',
        center=dict(lon=25, lat=30)
    ),
    height=700
)

fig_map.show()
fig_map.write_html('02_martyrdoms_map.html')
print("Map saved as '02_martyrdoms_map.html'")

Martyrdoms by Location:
                  Location  Deaths
10            Roman Empire    1200
11                    Rome     108
1                  Armenia     100
9           Persian Empire     100
7              Lyon (Gaul)      48
2                 Carthage       5
6   Jerusalem/Edessa/India       3
12           Rome/Carthage       3
5                Jerusalem       2
0               Alexandria       1
3                 Ethiopia       1
4               Hieropolis       1
8           Patmos/Ephesus       1
13                  Smyrna       1


Map saved as '02_martyrdoms_map.html'


## Density Heatmap

## Regional Analysis

In [ ]:
# Define regions
def assign_region(location):
    location = str(location).lower()
    if any(x in location for x in ['rome', 'lyon', 'carthage']):
        return 'Western Roman Empire'
    elif any(x in location for x in ['jerusalem', 'alexandria', 'ephesus', 'smyrna', 'hieropolis', 'nicomedia']):
        return 'Eastern Mediterranean'
    elif any(x in location for x in ['ethiopia', 'india']):
        return 'Africa & Asia'
    elif any(x in location for x in ['persia', 'armenia']):
        return 'Persia & Armenia'
    else:
        return 'Roman Empire (General)'

df['Region'] = df['Location'].apply(assign_region)

region_summary = df.groupby('Region').agg({
    'Deaths': 'sum'
}).sort_values('Deaths', ascending=False)

print("\nMartydoms by Region:")
print(region_summary)

# Pie chart
fig_region = px.pie(
    region_summary.reset_index(),
    values='Deaths',
    names='Region',
    title='Distribution of Martyrdoms by Region',
    color_discrete_sequence=px.colors.sequential.Reds,
    height=600
)

fig_region.show()
fig_region.write_html('04_martyrdoms_by_region.html')
print("Chart saved as '04_martyrdoms_by_region.html'")


Martydoms by Region:
                        Deaths
Region                        
Roman Empire (General)    1200
Persia & Armenia           200
Western Roman Empire       164
Eastern Mediterranean        9
Africa & Asia                1


Chart saved as '04_martyrdoms_by_region.html'


## Key Findings

### Notable Observations:

1. **Diocletian's Great Persecution (303 AD)** - Accounts for ~45% of all recorded martyrdoms
2. **Geographic Concentration** - Most persecutions occurred in the Eastern Mediterranean and Rome
3. **Temporal Pattern** - Sporadic persecutions in 1st-3rd centuries, intensifying under Diocletian
4. **Constantine's Impact** - 313 AD Edict of Milan marked the end of systematic Roman persecution
5. **Regional Variation** - Persecution continued outside Roman Empire (Persia, Armenia) after 313 AD

### Data Limitations:

- Estimates based on historical records and church tradition
- Exact dates and death tolls for large persecutions are uncertain
- Some figures have legendary elements mixed with historical accounts
- Represents only documented martyrdoms; actual totals likely much higher

In [ ]:
# Export data to CSV
df.to_csv('early_church_martyrdoms.csv', index=False)
print("Data exported to 'early_church_martyrdoms.csv'")

Data exported to 'early_church_martyrdoms.csv'
